In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [11]:
q = """
SELECT
    *
FROM
    custom.cth_renewal_base_static
"""

df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [12]:
df.insert(5,'static_base_2627',[10325,2292])

FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "cth_renewal_base_static")

df

,ticket_type,static_base_2223,static_base_2324,static_base_2425,static_base_2526,static_base_2627,processed_date
0,Full,4286,6134,7339,9554,10325,2026-01-15 18:59:20.234040+00:00
1,Premier,2857,2823,2991,3003,2292,2026-01-15 18:59:20.234040+00:00
